# run pyspark in jupyter

In [1]:
import findspark
findspark.init()

from pyspark import SparkContext
sc = SparkContext("local", "uspto_pyspark")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/12 00:25:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
input_file = "/Users/vincentcheng/Downloads/apc221127.xml"

from pyspark.sql import SparkSession
from pyspark.sql import functions as f

spark = SparkSession.builder.config("spark.sql.warehouse.dir").appName("uspto_preprocess").getOrCreate()

df_ori = spark.read.format("com.databricks.spark.xml").option("rowTag","case-file").load(input_file)



22/12/12 00:26:10 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
df_ori.printSchema()

root
 |-- case-file-header: struct (nullable = true)
 |    |-- abandonment-date: long (nullable = true)
 |    |-- amend-to-register-date: long (nullable = true)
 |    |-- amended-to-44d-application-in: string (nullable = true)
 |    |-- amended-to-44e-application-in: string (nullable = true)
 |    |-- amended-to-itu-application-in: string (nullable = true)
 |    |-- amended-to-use-application-in: string (nullable = true)
 |    |-- attorney-docket-number: string (nullable = true)
 |    |-- attorney-name: string (nullable = true)
 |    |-- cancellation-code: string (nullable = true)
 |    |-- cancellation-date: long (nullable = true)
 |    |-- cancellation-pending-in: string (nullable = true)
 |    |-- certification-mark-in: string (nullable = true)
 |    |-- change-registration-in: string (nullable = true)
 |    |-- collective-membership-mark-in: string (nullable = true)
 |    |-- collective-service-mark-in: string (nullable = true)
 |    |-- collective-trademark-in: string (nullable = 

In [22]:
df_ori.select(
    "*",
    f.explode("case-file-statements.case-file-statement")
).filter(
    f.substring(f.col('col.type-code'), 0, 2) == 'GS'
).select(
    f.col("serial-number").alias("application_number"),
    "case-file-header.filing-date",
    f.column("col.text").alias("gs"),
    f.substring(f.col("col.type-code"), 4, 2).alias("cls")
).filter(
    f.col("filing-date")>="19900101"
).select(
    "application_number","cls","gs"
).show()




+------------------+---+--------------------+
|application_number|cls|                  gs|
+------------------+---+--------------------+
|          75916583| 35|Managing hotels a...|
|          75916583| 43|RENTING OF TEMPOR...|
|          76483458| 05|Human vaccine pre...|
|          77659455| 43|Hotel, bar and re...|
|          77659460| 43|Hotel, bar and re...|
|          77911652| 43|Hotel, bar and re...|
|          86081025| 43|hotel, bar and re...|
|          86299351| 09|Protective cases ...|
|          86310386| 37|Management of rea...|
|          86310386| 36|Real estate servi...|
|          86310386| 35|Market research, ...|
|          86745892| 43|hotels, hotel res...|
|          87112551| 44|Farming services ...|
|          87601142| 05|Dietary and nutri...|
|          87601142| 35|Retail services p...|
|          87626338| 43|HOTEL, BAR AND RE...|
|          88015783| 43|HOTEL, BAR AND RE...|
|          90021244| 43|hotel, bar and re...|
|          90172577| 25|jackets, p

22/12/12 00:51:39 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 151479 ms exceeds timeout 120000 ms
22/12/12 00:51:39 WARN SparkContext: Killing executors is not supported by current scheduler.


In [112]:
# df1 - Phase 1: explode the necessary array
df1 = df.withColumn(
    'company',f.explode_outer(f.col('company'))
).withColumn(
   'emailAddress',f.explode_outer(f.col('emailAddress'))
).withColumn(
   'industryClassification',f.explode_outer(f.col('industryClassification'))
).withColumn(
   'physicalAddress',f.explode_outer(f.col('physicalAddress'))
)
# select the valid physical address
df1 = df1.filter(df1.physicalAddress.endDate.isNull())

# select the valid registered address
df1 = df1.withColumn(
   'registeredAddress',f.explode_outer(f.col('registeredAddress'))
)
df1 = df1.filter(df1.registeredAddress.endDate.isNull())

df1 = df1.withColumn(
   'tradingNames',f.explode_outer(f.col('tradingNames'))
)


In [113]:
df1.count()

998

In [101]:
def drop_col(df, struct_nm, delete_struct_child_col_nms):
    fields_to_keep = filter(lambda x:  x not in delete_struct_child_col_nms, df.select("{}.*".format(struct_nm)).columns)
    fields_to_keep = list(map(lambda x:  "{}.{}".format(struct_nm, x), fields_to_keep))
    return df.withColumn(struct_nm, f.struct(fields_to_keep))

In [114]:
# df2 - Phase 2: drop the unneeded columns
df2 = drop_col(df1, "company", ["shareholding", "stockExchangeListed"])
df2= df2.drop("historicShareholder", "insolvencyDetails", "roles", "ultimateHoldingCompany")

In [115]:
df2.printSchema()

root
 |-- australianBusinessNumber: string (nullable = true)
 |-- australianCompanyNumber: string (nullable = true)
 |-- australianServiceAddress: string (nullable = true)
 |-- company: struct (nullable = false)
 |    |-- annualReturnFilingMonth: string (nullable = true)
 |    |-- annualReturnLastFiled: string (nullable = true)
 |    |-- countryOfOrigin: string (nullable = true)
 |    |-- extensiveShareholding: string (nullable = true)
 |    |-- financialReportFilingMonth: string (nullable = true)
 |    |-- hasConstitutionFiled: string (nullable = true)
 |    |-- nzsxCode: string (nullable = true)
 |    |-- overseasCompany: string (nullable = true)
 |-- emailAddress: struct (nullable = true)
 |    |-- emailAddress: string (nullable = true)
 |    |-- emailPurpose: string (nullable = true)
 |    |-- emailPurposeDescription: string (nullable = true)
 |    |-- startDate: string (nullable = true)
 |    |-- uniqueIdentifier: string (nullable = true)
 |-- entityName: string (nullable = true)
